# Classificando os alimentos

In [14]:
import pandas as pd
from transformers import pipeline

# Load the data
data = pd.read_csv('tbalimentos.csv')

# Initialize the zero-shot classification pipeline
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# Function to classify food into breakfast, lunch, and dinner
def classify_food(item):
    labels = ['breakfast', 'lunch', 'dinner']
    result = classifier(item, candidate_labels=labels, multi_label=True)
    # Create a dictionary with classification results, initializing scores to 0
    label_scores = {label: 0 for label in labels}
    for label, score in zip(result['labels'], result['scores']):
        label_scores[label] = score
    return label_scores

# Apply classification to the text
data['Classification'] = data['NomeAlimento'].apply(classify_food)

# Function to check if any label scores are above a certain threshold
def assign_label_based_on_score(label_scores, threshold=0.1):  # Adjust threshold as needed
    return {label: score >= threshold for label, score in label_scores.items()}

# Use a low threshold to determine which labels should be true
data['Classification'] = data['Classification'].apply(assign_label_based_on_score)

# Create boolean columns based on classification results
data['IsBreakfast'] = data['Classification'].apply(lambda x: x['breakfast'])
data['IsLunch'] = data['Classification'].apply(lambda x: x['lunch'])
data['IsDinner'] = data['Classification'].apply(lambda x: x['dinner'])

# Drop the 'Classification' column if you no longer need it
data.drop(columns=['Classification'], inplace=True)

# Save the updated DataFrame to a new CSV file
data.to_csv('updated_tbalimentos.csv', index=False)

# Print the first few rows to check the output
print(data[['NomeAlimento', 'IsBreakfast', 'IsLunch', 'IsDinner']].head())

                NomeAlimento  IsBreakfast  IsLunch  IsDinner
0        Rice, brown, cooked         True     True      True
1       Rice, type 1, cooked         True     True      True
2       Rice, type 2, cooked         True     True      True
3          Oats, flakes, raw         True    False     False
4  Cookie, sweet, cornstarch         True    False     False


# Interpolando os requisitos

In [5]:
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d

def interpolate_dataset(file_path):
    # Load the dataset
    data = pd.read_csv(file_path)
    
    # Extract unique genders
    genders = data['Sexo'].unique()

    # Setup weight range for interpolation
    weight_range = np.arange(2, 636)  # Generates weights from 2 kg to 635 kg in 1 kg increments

    # Prepare a new DataFrame to hold the interpolated values
    interpolated_data = pd.DataFrame()

    for gender in genders:
        gender_data = data[data['Sexo'] == gender]
        
        # Create an interpolation function for each nutritional factor
        interpolated_funcs = {
            column: interp1d(gender_data['Peso'], gender_data[column], kind='linear', fill_value='extrapolate')
            for column in gender_data.columns if column != 'Sexo' and column != 'Peso'
        }
        
        # Use the interpolation functions to calculate values for the new weight range
        new_data = {column: func(weight_range) for column, func in interpolated_funcs.items()}
        new_data = {k: np.round(v, 1) for k, v in new_data.items()}  # Round each interpolated value to 1 decimal place
        new_data['Peso'] = weight_range
        new_data['Sexo'] = [gender] * len(weight_range)
        
        # Append to the main DataFrame
        interpolated_data = pd.concat([interpolated_data, pd.DataFrame(new_data)], ignore_index=True)

    return interpolated_data

# Usage example:
file_path = 'requisitos.csv'  # Change this to the path of your CSV file
interpolated_data = interpolate_dataset(file_path)
interpolated_data.to_csv('interpolated_data.csv', index=False)  # Save the output to a new CSV file